In [0]:
!pip install PyDrive
!pip install rouge
#!pip install pytorch-pretrained-bert
!pip install transformers

     |████████████████████████████████| 501kB 2.8MB/s 
     |████████████████████████████████| 3.7MB 53kB/s 
     |████████████████████████████████| 870kB 32.5MB/s 
     |████████████████████████████████| 1.0MB 34.1MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.38-cp36-none-any.whl size=884628 sha256=e2aa916d726be2504e7dc1e6b1f2b8e7868edfa03da0d340bce1238f9e73137b
  Stored in directory: /root/.cache/pip/wheels/6d/ec/1a/21b8912e35e02741306f35f66c785f3afe94de754a0eaf1422
Successfully built sacremoses


In [0]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
import json, gzip
import numpy as np
import time
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from google.colab import drive
from oauth2client.client import GoogleCredentials
import pickle
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence

#------------------------------------------------------------------------
!pip3 install pytorch-transformers
from transformers.modeling_utils import PreTrainedModel


# pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM, BasicTokenizer

from transformers import AlbertModel, AlbertTokenizer, AlbertForMaskedLM, AlbertConfig
#------------------------------------------------------------------------
import logging

     |████████████████████████████████| 184kB 2.8MB/s 


In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [0]:
print(torch.__version__)

1.4.0


In [0]:
from google.colab import auth
from pydrive.auth import GoogleAuth
from oauth2client.client import GoogleCredentials
from googleapiclient.discovery import build
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [0]:
!pip3 install pytorch-transformers

In [0]:
download = drive.CreateFile({'id': '1QZvEiZeKtyuDFxkuWL_Jbv_GpQTlKIco'})
download.GetContentFile('train.jsonl.gz')

In [0]:
#tokenizer = AutoTokenizer.from_pretrained("albert-xxlarge-v2")
#model = AutoModelWithLMHead.from_pretrained("albert-xxlarge-v2")

#tokenizer = AlbertTokenizer.from_pretrained('albert-base-v2')
tokenizer = AlbertTokenizer.from_pretrained('albert-large-v2')

#model = AlbertModel.from_pretrained('albert-base-v2')
model = AlbertModel.from_pretrained('albert-large-v2')
input_ids = torch.tensor(tokenizer.encode("Hello, my dog is cute", add_special_tokens=True)).unsqueeze(0)  # Batch size 1
outputs = model(input_ids)
last_hidden_states = outputs[0]


path = "train.jsonl.gz"
data = []
with gzip.open(path) as f:
    for ln in f:
        t = json.loads(ln.decode('utf-8'))
        text = t['text']
        tokenized_text = tokenizer.tokenize(text).copy() 
        l = len(tokenized_text)
        if l > 512:
            continue
        print(l)  

       
        summ = t['summary']
        tokenized_summ = tokenizer.tokenize(summ)
        s = len(tokenized_summ)
        if s > 512:
            continue
        print(s)    
        data.append(t)
        if(len(data)>=100):
            break
#print('......')           
#print(len(data))
train_dict = data



396
114
410
93
385
100
172
106
161
13
178
15
506
106
457
95
400
90
161
37
384
50
338
14
120
15
314
14
251
23
497
22
495
133
148
37
361
12
494
146
396
27
389
26
305
145
76
32
197
146
392
114
253
20
113
95
224
27
287
29
423
19
215
34
266
10
221
28
262
24
273
21
410
9
246
19
401
9
370
10
382
64
286
27
258
18
262
22
296
23
431
39
408
72
472
18
373
15
245
21
244
29
318
36
135
7
424
26
336
24
393
32
232
16
214
13
201
35
509
31
134
24
320
16
350
16
457
21
393
20
389
71
245
22
357
29
283
28
442
36
419
78
243
18
502
35
203
15
251
77
212
13
428
17
473
29
365
23
401
13
314
91
274
15
387
17
440
57
423
80
490
26
437
24
297
77
338
18
330
159
252
15
484
22
350
17
251
21
112
36
342
56
438
12
178
13
337
6
222
49


In [0]:
print(train_dict[0]['text'])

Spinach has terrorized generations of veggie-phobic kids, and many grownups don't much like it, either. But when it's combined with seasonings and feta cheese and wrapped in a golden crisp phyllo dough crust, even those who despise Popeye's Â­favorite food ask for seconds.

The spinach pie at Kebab House II on Orchard St. is a specialty of owner and executive chef Ramazan Ay, who serves the hearty dish as a main course. You can make individual tarts or one large pie, though the small ones are more labor-intensive. If you've never worked with phyllo dough before, know that it is important to keep the part you are not using covered with a damp towel so it won't dry out. Other than that, this dish is a snap to make - and a favorite in my family, even among those who eschew anything green.

1 onion, peeled and chopped

2 cups crumbled feta cheese

Salt and pepper to taste

1 pound phyllo dough, thawed if frozen

Preheat the oven to 400 degrees. In a sautÃ© pan, heat the olive oil over low 

In [0]:
train_dict = sorted(train_dict, key = lambda i: len(i['text']), reverse = True)

In [0]:
mx = 0
count  = 0
for t in train_dict:
    text = t['text']
    tokenized_text = tokenizer.tokenize(text).copy()
    l = len(tokenized_text)
    if l > mx:
        mx = l
    summ = t['summary']
    tokenized_summ = tokenizer.tokenize(summ).copy()
    s = len(tokenized_summ)
    if s> count:
        count = s
print(mx)
print(count)

509
159


In [0]:
#from pytorch_pretrained_bert import BertTokenizer, BertModel, BertForMaskedLM

In [0]:
mx = 0
count  = 0

torch.cuda.empty_cache() 

text_seq_len = 509
s_seq_len = 159

#text_seq_len = 512
#s_seq_len = 160

batch_size = 100
text_tensor = torch.zeros([batch_size, text_seq_len ], dtype=torch.int64)
summ_tensor = torch.zeros([batch_size, s_seq_len ], dtype=torch.int64)
pad_len_tensor = torch.zeros([batch_size], dtype=torch.int64)
i = 0
for t in train_dict:
#   print(t)
    text = t['text']
    tokenized_text = tokenizer.tokenize(text)
    l = len(tokenized_text)
    summ = t['summary']
    tokenized_summ = tokenizer.tokenize(summ)
    s = len(tokenized_summ)
    if l>=s:
        if l<text_seq_len:
            pad = ['[PAD]' ] * (text_seq_len-l)
            tokenized_text += pad 
            pad_len_tensor[i] = text_seq_len-l
        if s<s_seq_len:
            pad = ['[PAD]' ] * (s_seq_len-s)
            tokenized_summ += pad
        if s > mx:
            mx = s
        indexed_tokens_t = tokenizer.convert_tokens_to_ids(tokenized_text) 
        indexed_tokens_s = tokenizer.convert_tokens_to_ids(tokenized_summ)
        tokens_tensor_t = torch.tensor(indexed_tokens_t)
        text_tensor[i] =  tokens_tensor_t
        tokens_tensor_s = torch.tensor(indexed_tokens_s)
        summ_tensor[i] = tokens_tensor_s
        i+=1



#model = AlbertModel.from_pretrained('albert-base-v2')

model = AlbertModel.from_pretrained('albert-large-v2')


model.eval()  
text_tensor = text_tensor.long().to('cuda')
summ_tensor = summ_tensor.long().to('cuda')
pad_len_tensor = pad_len_tensor.long().to('cuda')
model.to('cuda')

#torch.cuda.empty_cache() 


with torch.no_grad():
    
    encoded_layers_t, _ = model(text_tensor) #, output_all_encoded_layers=False)
    encoded_layers_s, _ = model(summ_tensor) #, output_all_encoded_layers=False)

print(encoded_layers_t.size())
print(encoded_layers_s.size()) 
print(pad_len_tensor.size())
print(mx)

torch.Size([100, 509, 1024])
torch.Size([100, 159, 1024])
torch.Size([100])
159


In [0]:
import torch.nn as nn

In [0]:
class biEncoderLSTM(nn.Module):
    
    def __init__(self, params):
        super(biEncoderLSTM, self).__init__()
        self.hidden_size = params['hidden_size']
        self.emb_size = params['emb_size']
        self.vocab_size = params['vocab_size']
        self.num_layers = params['num_layers']
        self.embedding = nn.Embedding(self.vocab_size, self.emb_size)
        self.lstm = nn.LSTM(self.emb_size, self.hidden_size, batch_first = True, num_layers=self.num_layers, bidirectional = True)

        print(self.hidden_size)


    def forward(self, input, hidden, memory, seq_lengths):
        batch, seq_len, emb_size = input.size()
        emb = input

        #print(emb)

#         packed_emb = pack_padded_sequence(emb, (seq_lengths).cpu().numpy(), batch_first = True)
#         packed_output, (h, c) = self.lstm(packed_emb, (hidden, memory))
#         output, _ = pad_packed_sequence(packed_output, batch_first = True)
        output, (h, c) = self.lstm(emb, (hidden, memory))
        output1 = output[ :, :, :self.hidden_size]
        output2 = output[:, :, self.hidden_size:] #reverse
        output = output1+output2
        h = h.view(self.num_layers, 2, batch, self.hidden_size)
        c = c.view(self.num_layers, 2, batch, self.hidden_size)
        h = h[:,0,:,:]
        c = c[:,0,:,:]
        return output, h, c

    def initHidden(self, batch_size):
        h_init = torch.randn(self.num_layers*2, batch_size, self.hidden_size).cuda()
        c_init = torch.randn(self.num_layers*2, batch_size, self.hidden_size).cuda()
        return h_init, c_init

In [0]:
class DecoderLSTM(nn.Module):
    
    def __init__(self, params):
        super(DecoderLSTM, self).__init__()
        self.hidden_size = params['hidden_size']
        self.vocab_size = params['vocab_size']
        self.emb_size = params['emb_size']
        self.num_layers = params['num_layers']
        self.embedding = nn.Embedding(self.vocab_size, self.emb_size)
        self.lstm = nn.LSTM(self.emb_size, self.hidden_size, batch_first = True)
        self.w = nn.Linear(self.hidden_size, self.vocab_size)

    def forward(self, input, hidden, memory):
        batch_size, seq_len = input.size()
        emb = self.embedding(input)
        output, (h, c) = self.lstm(emb, (hidden, memory))
        logits = self.w(output)
        return logits, h, c

    def initHidden(self):
        h_init = torch.randn(self.num_layers, batch_size, self.d_hid).cuda()
        c_init = torch.randn(self.num_layers, batch_size, self.d_hid).cuda()
        return h_init, c_init

In [0]:
class AttnDecoderLSTM(nn.Module):
    
    def __init__(self, params):
        super(AttnDecoderLSTM, self).__init__()
        self.hidden_size = params['hidden_size']
        self.vocab_size = params['vocab_size']
        self.emb_size = params['emb_size']
        self.num_layers = params['num_layers']
        self.batch_size = params['batch_size']
        #self.dropout_p = params['dropout_p']
         
       

        self.embedding = nn.Embedding(self.vocab_size, self.hidden_size)
        #self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        #self.dropout = nn.Dropout(self.dropout_p)
        self.lstm = nn.LSTM(self.emb_size, self.hidden_size, batch_first=True, num_layers = self.num_layers)
        self.w = nn.Linear(2*self.hidden_size, self.vocab_size)

    def forward(self, input, hidden, memory, encoder_outputs, pad_ind):
        batch_size, seq_len, emb_size = input.size()
        embedded = input
        #embedded = self.dropout(embedded)
        
        output,(h,c) = self.lstm(embedded, (hidden, memory))
        
        attn_values = torch.bmm(output,encoder_outputs.permute(0,2,1))
        dec_attn_values = torch.bmm(output,output.permute(0,2,1))
    
        for b in range(batch_size):
            attn_values[b,:,pad_ind[b]:] = -1e10
        
        mask = torch.ones_like(dec_attn_values).cuda()
        mask1 = -1e10*torch.ones_like(dec_attn_values).cuda()
        
        mask = torch.tril(mask, diagonal = -1)
        mask1 = torch.triu(mask1)
        mask = mask+mask1
        dec_attn_values = dec_attn_values*mask.float()
        
        total_attn_values = torch.cat((attn_values,dec_attn_values),2)
        total_outputs = torch.cat((encoder_outputs,output),1)
        
        sftmax = nn.Softmax(dim=2)
        total_attn_weights = sftmax(total_attn_values)
        weight_attn = torch.bmm(total_attn_weights, total_outputs)
        final_hidden = torch.cat((output,weight_attn),2)
        scores = self.w(final_hidden)
        return scores, h, c


    def initHidden(self, batch_size):
        h_init = torch.randn(self.num_layers, batch_size, self.hidden_size).cuda()
        c_init = torch.randn(self.num_layers, batch_size, self.hidden_size).cuda()
        return h_init, c_init

In [0]:
from rouge import Rouge

def calculate_rouge (hyps, refs):
    #print(hyps, refs)
    r = Rouge()
    scores = r.get_scores(hyps, refs, avg=True)
    return(scores)


In [0]:
import torch

In [0]:
def train_lm(encoded_layers_t, encoded_layers_s, pad_len_tensor, summ_tensor, params, net, fscore = True):
    
    # since the first index corresponds to the PAD token, we just ignore it
    # when computing the loss
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    sftmax = nn.Softmax(dim=2)
    optimizer = optim.Adam(net.parameters(), lr=params['learning_rate'])
    num_examples = encoded_layers_t.size()[0]    
    batches = [(start, start + params['batch_size']) for start in range(0, num_examples, params['batch_size'])]
    
    for epoch in range(params['epochs']):
        ep_loss = 0.
        start_time = time.time()
        #random.shuffle(batches)
        ref_summ = []
        gen_summ = []
        count = 0
        # for each batch, calculate loss and optimize model parameters            
        for b_idx, (start, end) in enumerate(batches):
            batch_text = encoded_layers_t[start:end]
            batch_summary = encoded_layers_s[start:end]
            batch_pad_len = pad_len_tensor[start:end]
            tar = summ_tensor[start:end]

            
            pred = net(batch_text, batch_summary, batch_pad_len)
            preds = pred[:, :-1, :].contiguous().view(-1, net.vocab_size)
            
            targets = tar[:, 1:].contiguous().view(-1)
            loss = criterion(preds, targets)
            
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            ep_loss += loss
        
        if fscore:
                logits = sftmax(pred)
                words = torch.argmax(logits, dim=2)
                for sent in range(words.size()[0]):
                    summ = ''
                    gold = ''
                    for word in range(words.size()[1]):
                        if tokenizer.convert_ids_to_tokens([words[sent][word].item()])[0] == 'EOS':
                            break
                        if summ :
                            summ += ' '
                        summ += tokenizer.convert_ids_to_tokens([words[sent][word].item()])[0]

                    for word in range(1,len(summ_tensor[sent])-1):
                        if tokenizer.convert_ids_to_tokens([summ_tensor[sent][word].item()])[0] == 'EOS':
                            break
                        if gold:
                            gold += ' '
                        gold += tokenizer.convert_ids_to_tokens([summ_tensor[sent][word].item()])[0]

                    if summ == '.'*len(summ):
                        count += 1
                    if not summ or summ == '.'*len(summ):
                        summ = '====='

                    gen_summ.append(summ)
                    ref_summ.append(gold)
        
        ind = 0
        scores = pred[ind,:,:].unsqueeze(0)
        logits = sftmax(scores)
        words = torch.argmax(logits, dim=2).squeeze(0)
        gold_summ_last = ''
        summ_last = ''
        for l in range(summ_tensor[ind,1:].size()[0]):
            gold_summ_last += tokenizer.convert_ids_to_tokens([summ_tensor[ind,l].item()])[0] + ' '
        for l in range(words.size()[0]):
            summ_last += tokenizer.convert_ids_to_tokens([words[l].item()])[0] + ' '
        print('########################Train######################')
        print(gold_summ_last)
        print(summ_last)
        
        if fscore:
            rouge_score = calculate_rouge(gen_summ, ref_summ)
            print('rouge', rouge_score)
        print('epoch: %d, loss: %0.2f, time: %0.2f sec' %\
              (epoch, ep_loss, time.time()-start_time))
        print('####################################################')

In [0]:
# function to evaluate LM perplexity on some input data, DO NOT MODIFY
def compute_loss(dataset, net, bsz=10, fscore = False):
    
    sftmax= nn.Softmax(dim=2)
    criterion = nn.CrossEntropyLoss(ignore_index=0)
    num_examples = len(dataset)  
    # we'll still use batches b/c we can't fit the whole
    # validation set into GPU memory
    batches = [(start, start + bsz) for start in range(0, num_examples, bsz)]
    
    total_unmasked_tokens = 0. # count how many unpadded tokens there are
    nll = 0.
    gen_summ = []
    ref_summ = []
    
    for b_idx, (start, end) in enumerate(batches):
            
        batch = dataset[start:end]
        batch_text = np.zeros([len(batch), len(batch[0]['text'])])
        batch_summary = np.zeros([len(batch), len(batch[0]['summary'])])
        batch_pad_len = np.zeros([len(batch)])
        i = 0
        for d in batch:
            batch_text[i] = d['text']
            batch_summary[i] = d['summary']
            batch_pad_len[i] = d['text_len']
            i += 1
        batch_text = torch.from_numpy(batch_text).long().cuda()
        batch_summary = torch.from_numpy(batch_summary).long().cuda()
        batch_pad_len = torch.from_numpy(batch_pad_len).long().cuda()
            
        ut = torch.nonzero(batch_text).size(0)
        pred = net(batch_text, batch_summary, batch_pad_len)
        targets = batch_summary[:, 1:].contiguous().view(-1)
        preds = pred[:, :-1, :].contiguous().view(-1, net.vocab_size)
        loss = criterion(preds, targets)
        nll += loss.detach()
        total_unmasked_tokens += ut
        
        if fscore:
            #print('pred', pred.size())
            logits = sftmax(pred)
            words = torch.argmax(logits, dim=2)
            #print('words', words.size())
            for sent in range(words.size()[0]):
                summ = ''
                gold = ''
                for word in range(words.size()[1]):
                    if index2word[words[sent][word].item()] == 'EOS':
                        break
                    if summ :
                        summ += ' '
                    summ += index2word[words[sent][word].item()]

                for word in range(1,len(batch_summary[sent])-1):
                    if index2word[batch_summary[sent][word].item()] == 'EOS':
                        break
                    if gold:
                        gold += ' '
                    gold += index2word[batch_summary[sent][word].item()]

                if summ == '.'*len(summ):
                    count += 1
                if not summ or summ == '.'*len(summ):
                    summ = '====='

                gen_summ.append(summ)
                ref_summ.append(gold)
        
    
    scores = pred[14,:,:].unsqueeze(0)
    logits = sftmax(scores)
    words = torch.argmax(logits, dim=2).squeeze(0)
    gold_summ = ''
    summ = ''
    for l in range(batch_summary[14,1:].size()[0]):
        gold_summ += index2word[batch_summary[14,l].item()] + ' '
    for l in range(words.size()[0]):
        summ += index2word[words[l].item()] + ' '
            
    
    print('#############Dev###########')
    if fscore:
        rouge_score = calculate_rouge(gen_summ, ref_summ)
        print('rouge', rouge_score)
    print(gold_summ)
    print(summ)
    
    perplexity = torch.exp(nll / total_unmasked_tokens).cpu()
    return nll, perplexity.data


In [0]:
class SuperAwesome(nn.Module):
    def __init__(self, params):
        super(SuperAwesome, self).__init__()
        #self.enc = EncoderLSTM(params).cuda()
        self.bienc = biEncoderLSTM(params).cuda()
        #self.dc = DecoderLSTM(params).cuda()
        self.attn_dc = AttnDecoderLSTM(params).cuda()
        self.vocab_size = params['vocab_size']
    def forward(self, batch_text, batch_summary, batch_pad_len):
        batch_size, seq_size, emd_size = batch_text.size()

#         h_init,c_init = self.enc.initHidden(batch_size)
#         enc_output,h_enc,c_enc = self.enc(batch_text,h_init,c_init, batch_pad_len)
        
        h_init,c_init = self.bienc.initHidden(batch_size)
        enc_output,h_enc,c_enc = self.bienc(batch_text,h_init,c_init, batch_pad_len)
        #h_init,c_init = self.attn_dc.initHidden(batch_size)
        h_init, c_init = h_enc.contiguous(),c_enc.contiguous()
        attn_pre,h,c = self.attn_dc(batch_summary,h_init,c_init,enc_output, batch_pad_len)
#         attn_pre, h, c = self.dc(batch_summary, h_enc, c_enc)
        return attn_pre



#train_lm(encoded_layers_t, encoded_layers_s, pad_len_tensor, summ_tensor, params, net, fscore = True):


#print(encoded_layers_t)
#print(encoded_layers_s)
#print(pad_len_tensor)
#print(summ_tensor)
#print(params)
#print(net)
#print(fscore)


In [0]:
params = {}
params['hidden_size'] =  512


# params['emb_size'] = 2048
#params['emb_size'] = 768
params['emb_size'] = 1024

params['num_layers'] = 2
params['batch_size'] = 10

params['vocab_size'] = 30000

params['learning_rate'] = 0.001
params['epochs'] =100

net = SuperAwesome(params)

train_lm(encoded_layers_t, encoded_layers_s,pad_len_tensor, summ_tensor, params, net, fscore = True)


#train_lm(encoded_layers_t, encoded_layers_s, pad_len_tensor, summ_tensor, params, net, fscore = True):

512
########################Train######################
▁most ▁boomer ▁and ▁gen ▁x ▁women ▁wanted ▁to ▁prior it ize ▁work , ▁but ▁ended ▁up ▁focused ▁on ▁child - rea ring . ▁can ▁millennia ls ▁avoid ▁the ▁same ▁trap ? <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> <unk> 
<unk> <unk> <unk> <